# Preparation

In [1]:
import os
import cv2
import shutil
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, Video

In [ ]:
OUTPUT_PATH = "/kaggle/working"
DATASET_PATH = "/kaggle/input/snooker-balls/balls"
dataset_path = "/kaggle/working/yolo_dataset"

# Dataset 

### Original Dataset Structure
Dataset structure is given in next format:
```
snooker-balls/balls/
    train/
        {class1}/
            {id_1}.jpg
            {id_2}.jpg
            ...
        {class2}/
            ...
        ...
    test/
    ...
```

### Yolo Dataset Structure
Dataset structure should be transformed to next format:
```
yolo_dataset/
    images/
        train/
            {class1}_{id_1}.jpg
            {class1}_{id_2}.jpg
            ...
        val/
            ...
    labels/
        train/
            {class1}_{id_1}.txt
            {class1}_{id_2}.txt
            ...
        val/
            ...
```

In [ ]:
# root directory to save dataset in yolo format
os.makedirs(dataset_path, exist_ok=True)

os.makedirs(f'{dataset_path}/images', exist_ok=True)
os.makedirs(f'{dataset_path}/images/train', exist_ok=True)
os.makedirs(f'{dataset_path}/images/val', exist_ok=True)

os.makedirs(f'{dataset_path}/labels', exist_ok=True)
os.makedirs(f'{dataset_path}/labels/train', exist_ok=True)
os.makedirs(f'{dataset_path}/labels/val', exist_ok=True)

In [ ]:
# variables to convert label/id to id/label
label2id = {"black": 0, "blue": 1, "brown": 2, "green": 3, "pink": 4, "red": 5, "white": 6, "yellow": 7}
id2label = {v: k for k, v in label2id.items()}

In [ ]:
for layer1 in os.listdir(DATASET_PATH):
    for layer2 in os.listdir(f'{DATASET_PATH}/{layer1}'):
        for layer3 in os.listdir(f'{DATASET_PATH}/{layer1}/{layer2}'):
            if layer3.endswith('.jpg'):
                shutil.copyfile(f'{DATASET_PATH}/{layer1}/{layer2}/{layer3}',f'{dataset_path}/images/{layer1}/{layer2}_{layer3}'.replace('test', 'val' ))
                f = open(f'{dataset_path}/labels/{layer1}/{layer2}_{layer3}'.replace('test', 'val' ).replace('.jpg', '.txt' ), 'w')
                f.write(f'{label2id[layer2]} 0.5 0.5 1 1')
                f.close()

# YOLOv8

In [ ]:
!pip install -U ultralytics
!yolo checks
from ultralytics import YOLO

### Convert Dataset to Yolo Format

In [ ]:
names_content = "\n".join([f"  {label_id}: {label}" for label, label_id in label2id.items()])
dataset_content = f"""
path: "{dataset_path}/"
train: "images/train"
val: "images/val"
names:
{names_content}
"""
with open(os.path.join(OUTPUT_PATH, "custom_dataset.yaml"), "w") as f:
    f.write(dataset_content)

### Train the Model

In [ ]:
# pretrained model: yolov8n、yolov8s、yolov8m、yolov8l、yolov8x
model = YOLO('yolov8n.yaml').load('yolov8n.pt')

# Train the model using the processed dataset
results = model.train(
    data=os.path.join(OUTPUT_PATH,'custom_dataset.yaml'),
    project='snooker_project',
    exist_ok=True,
    epochs=4,
    batch=8,
    imgsz=16,
    optimizer='Adam',
    lr0=0.001,
    lrf=0.0005
)

### Validate the Model

In [ ]:
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

### Export the Model with **ONNX**

In [ ]:
model.export(format='onnx')

### Benchmark the Model with **ONNX**

In [ ]:
from ultralytics.utils.benchmarks import benchmark
benchmark(model='snooker_project/train/weights/best.pt', imgsz=128)

### Create a Download Link

In [34]:
%cd $OUTPUT_PATH
!zip -r snooker_project.zip snooker_project
from IPython.display import FileLink
FileLink(r'snooker_project.zip')

/kaggle/working
  adding: snooker_project/ (stored 0%)
  adding: snooker_project/train/ (stored 0%)
  adding: snooker_project/train/val_batch1_pred.jpg (deflated 35%)
  adding: snooker_project/train/weights/ (stored 0%)
  adding: snooker_project/train/weights/best.pt (deflated 56%)
  adding: snooker_project/train/weights/last.pt (deflated 56%)
  adding: snooker_project/train/events.out.tfevents.1703947480.ab3b6a412c0b.42.0 (deflated 9%)
  adding: snooker_project/train/val_batch2_labels.jpg (deflated 34%)
  adding: snooker_project/train/train_batch1.jpg (deflated 5%)
  adding: snooker_project/train/labels.jpg (deflated 54%)
  adding: snooker_project/train/val_batch1_labels.jpg (deflated 37%)
  adding: snooker_project/train/args.yaml (deflated 52%)
  adding: snooker_project/train/R_curve.png (deflated 9%)
  adding: snooker_project/train/val_batch0_pred.jpg (deflated 28%)
  adding: snooker_project/train/val_batch2_pred.jpg (deflated 30%)
  adding: snooker_project/train/PR_curve.png (defla

/kaggle/working/snooker_project.zip

# Image from Test

In [ ]:
image = '/kaggle/input/snooker-private-test/image2.jpg'

In [ ]:
%cd $OUTPUT_PATH
best_model = YOLO("snooker_project/train/weights/best.pt")
best_model.predict(source=image, show=False, save=True, name='image', exist_ok=True)

In [ ]:
%cd $OUTPUT_PATH
plt.imshow(cv2.cvtColor(cv2.imread('runs/detect/image/image2.jpg'), cv2.COLOR_BGR2RGB))
plt.show()

# Video with PyTube

In [ ]:
!pip install pytube
from pytube import YouTube
YouTube('https://youtu.be/hw02UKK4Kb0').streams.filter().get_highest_resolution().download(output_path=OUTPUT_PATH, filename='youtube.mp4')

In [ ]:
%cd $OUTPUT_PATH
!ffmpeg -i youtube.mp4 -vcodec copy -acodec copy -ss 00:01:05 -to 00:01:15 video.mp4 -y

In [ ]:
%cd $OUTPUT_PATH
Video('video.mp4', width=840)

In [ ]:
#video = os.path.join(OUTPUT_PATH, 'video.mp4')
video = '/kaggle/input/snooker-private-test/video.mp4'

# Predict the Video

In [ ]:
%cd $OUTPUT_PATH
best_model = YOLO("snooker_project/train/weights/best.pt")
best_model.predict(source=video, show=False, save=True, name='predict', exist_ok=True)

In [ ]:
%cd $OUTPUT_PATH
!ffmpeg -y -loglevel panic -i runs/detect/predict/video.avi predict_video.mp4
Video("predict_video.mp4", width=840)

# Track the Video with **ByteTrack**

In [ ]:
%cd $OUTPUT_PATH
best_model = YOLO('snooker_project/train/weights/best.pt')
best_model.track(source=video, tracker="bytetrack.yaml", save=True, name='track', exist_ok=True)

In [ ]:
%cd $OUTPUT_PATH
!ffmpeg -y -loglevel panic -i runs/detect/track/video.avi track_video.mp4
Video("track_video.mp4", width=840)

# Predict with **SAHI**

In [ ]:
!pip install -U sahi

In [ ]:
from sahi.utils.yolov8 import download_yolov8s_model
from sahi import AutoDetectionModel
# from sahi.utils.cv import read_image
# from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
# from pathlib import Path

In [ ]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path="snooker_project/train/weights/best.pt",
    confidence_threshold=0.3,
    device="cpu",  # or 'cuda:0'
)

In [ ]:
result = get_sliced_prediction(
    "/kaggle/input/snooker-private-test/image2.jpg",
    detection_model,
    slice_height=50,
    slice_width=50,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2
)

In [ ]:
# best_model.predict(source=video, show=False, save=True, name='predict', exist_ok=True)
predict(
    model_type="yolov8",
    model_path="snooker_project/train/weights/best.pt",
    model_device="cpu",  # or 'cuda:0'
    model_confidence_threshold=0.4,
    source="/kaggle/input/snooker-private-test/video.mp4",
    slice_height=256,
    slice_width=256,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
)

In [ ]:
result.export_visuals(export_dir="demo_data/")
Image("demo_data/prediction_visual.png")